# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import json
import math
import statistics

# Upload Datasets

In [2]:
Data_File1 = pd.read_csv (r'/homedir01/eeldeeb19/Oulu Data/sc-export-2021022500-elsys.csv')

In [3]:
Data_File1.head()

,time,deveui,temperature,humidity,light,motion,co2,battery,sound_avg,sound_peak,moisture,pressure,acceleration_x,acceleration_y,acceleration_z
0,1596240002191,a8-17-58-ff-fe-04-64-6b,21.4,44.0,0.0,79.0,NaN,3.635,34.0,64.0,NaN,NaN,NaN,NaN,NaN
1,1596240006323,a8-17-58-ff-fe-04-63-af,21.5,46.0,0.0,86.0,NaN,3.638,34.0,64.0,NaN,NaN,NaN,NaN,NaN
2,1596240010305,a8-17-58-ff-fe-04-63-b0,21.5,47.0,0.0,76.0,NaN,3.638,34.0,64.0,NaN,NaN,NaN,NaN,NaN
3,1596240012968,a8-17-58-ff-fe-03-0f-71,21.5,45.0,0.0,5.0,500.0,3.684,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1596240013257,a8-17-58-ff-fe-03-10-45,21.5,47.0,0.0,5.0,479.0,3.686,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Extract Time Columns

In [4]:
Time_1 = Data_File1["time"]

# Decode Time per device for File 1

Here, we collect the readings of each device individullay to detect the failed transmission of each device.

In [5]:
Deveui_1 = Data_File1["deveui"]
Devices_1 = dict(Counter(Deveui_1))
len(Devices_1)
# print(a)

468

In [6]:
Devices_Headers_1 = list(Devices_1.keys())
# print(Devices_Headers_1)
Deveui_lis_1 = list(Deveui_1)

In [7]:
all_list_1 = []
for i in range (0,len(Devices_Headers_1)) :
    res_list_1 = []
    for j in range (0,len(Deveui_lis_1)) :
        if Deveui_lis_1[j] == Devices_Headers_1[i] : 
            res_list_1.append(j)
    all_list_1.append(res_list_1)
#print(Deveui_lis_1.index(it[256]))
print(all_list_1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
print(len(all_list_1))

468


In [12]:
print(len(Devices_Headers_1))

468


# Add Missing Data for Some Devices

We rely on the fact that each device should transmit every 15 minutes. Therefore, we detect the failed transmission whenever there is no transmission after 15 minutes from the last transmission.

We add a row of zeros whenever a failed transmission is detected. This row has the time stamp of the transmission that is missing and zeros in the parameters that should have been measured by the sensors.

In [13]:
Co2 = Data_File1["co2"]
temperature = Data_File1["temperature"]
humidity = Data_File1["humidity"]
light = Data_File1["light"]
motion = Data_File1["motion"]
battery = Data_File1["battery"]
sound_avg = Data_File1["sound_avg"]
sound_peak = Data_File1["sound_peak"]
moisture = Data_File1["moisture"]
pressure = Data_File1["pressure"]
TIME_1 = Data_File1["time"]
All_MISSES = 0
All_Lengths = 0

count = 0

for i in range (0,len(Devices_Headers_1)):
    t = all_list_1[i]
    Co2_i = Co2[t]
    Co2_i = [kk for kk in Co2_i if str(kk) != 'nan']
    if len(Co2_i) == 0:
        count = count + 1
    else:
        Co2_i = Co2[t]
        temperature_i = temperature[t]
        humidity_i = humidity[t]
        light_i = light[t]
        motion_i = motion[t]
        battery_i = battery[t]
        sound_avg_i = sound_avg[t]
        sound_peak_i = sound_peak[t]
        moisture_i = moisture[t]
        pressure_i = pressure[t]
        
        TIME_1_i = TIME_1[t]
        TIME_1_i = [x for x in TIME_1_i if str(x) != 'nan']
        TIME_1_i = np.array(TIME_1_i)
        
        #id_str = str(i)

        Device = np.stack((TIME_1_i,Co2_i,temperature_i,humidity_i,light_i,motion_i,battery_i,sound_avg_i,sound_peak_i,moisture_i,pressure_i),axis=-1)
        Device_df = pd.DataFrame(data = Device,columns = ['Time','Co2','Temperature','Humidity','Light','Motion','Battery','Sound_avg','Sound_peak','Moisture','Pressure'])
        Device_df.to_csv('Dev_' + Devices_Headers_1[i] + '.csv', index=True, header=True, sep=',')        
        
        
        New_Co2 = Co2_i.copy()
        New_Co2 = New_Co2.to_numpy()
        New_temperature = temperature_i.copy()
        New_temperature = New_temperature.to_numpy()
        New_humidity = humidity_i.copy()
        New_humidity = New_humidity.to_numpy()
        New_light = light_i.copy()
        New_light = New_light.to_numpy()
        New_motion = motion_i.copy()
        New_motion = New_motion.to_numpy()
        New_battery = battery_i.copy()
        New_battery = New_battery.to_numpy()
        New_sound_avg = sound_avg_i.copy()
        New_sound_avg = New_sound_avg.to_numpy()
        New_sound_peak = sound_peak_i.copy()
        New_sound_peak = New_sound_peak.to_numpy()
        New_moisture = moisture_i.copy()
        New_moisture = New_moisture.to_numpy()
        New_pressure = pressure_i.copy()
        New_pressure = New_pressure.to_numpy()
        
        Time_Diff_1_i = []


        for j in range(1,len(TIME_1_i)):
            Time_Diff_1_i.append(TIME_1_i[j]-TIME_1_i[j-1])

        Time_Diff_i_minutes = [l/(1000*60) for l in Time_Diff_1_i]
        EXTRA_TIME_IDX = [j for j in range(len(Time_Diff_i_minutes)) if(Time_Diff_i_minutes[j]>29.5)]

        MISSES = []
        EXTRA_NEW = EXTRA_TIME_IDX.copy()
        EXTRA_NEW.append(0)


        for j in range(0,len(EXTRA_TIME_IDX)):
            Num_Miss = math.floor(Time_Diff_i_minutes[EXTRA_TIME_IDX[j]]/14.75)-1
            MISSES.append(Num_Miss)
        
        NEW_IDX_2 = np.insert(np.cumsum(MISSES,axis=0),0,0)
        NEW_IDX = EXTRA_NEW+NEW_IDX_2
        NEW_IDX = NEW_IDX[0:len(NEW_IDX)-1]


        for j in range(0,len(EXTRA_TIME_IDX)):
            INSERTION = []
            INSERTION_CO2 = []
            for k in range(0,MISSES[j]):
                INSERTION.append(np.sum([TIME_1_i[NEW_IDX[j]],900000*(k+1)]))
                INSERTION_CO2.append(0)
            TIME_1_i = np.insert(TIME_1_i,NEW_IDX[j]+1,INSERTION)
            New_Co2 = np.insert(New_Co2,NEW_IDX[j]+1,INSERTION_CO2)
            New_temperature = np.insert(New_temperature,NEW_IDX[j]+1,INSERTION_CO2)
            New_humidity = np.insert(New_humidity,NEW_IDX[j]+1,INSERTION_CO2)
            New_light = np.insert(New_light,NEW_IDX[j]+1,INSERTION_CO2)
            New_motion = np.insert(New_motion,NEW_IDX[j]+1,INSERTION_CO2)
            New_battery = np.insert(New_battery,NEW_IDX[j]+1,INSERTION_CO2)
            New_sound_avg = np.insert(New_sound_avg,NEW_IDX[j]+1,INSERTION_CO2)
            New_sound_peak = np.insert(New_sound_peak,NEW_IDX[j]+1,INSERTION_CO2)
            New_moisture = np.insert(New_moisture,NEW_IDX[j]+1,INSERTION_CO2)
            New_pressure = np.insert(New_pressure,NEW_IDX[j]+1,INSERTION_CO2)
        
        MISS_NUM = np.zeros(shape=(len(New_Co2)))
        MISS_NUM[:] = np.NaN
        MISS_NUM[0] = np.sum(MISSES)
        Error = np.zeros(shape=(len(New_Co2)))
        Error[:] = np.NaN
        Error[0] = (MISS_NUM[0]/len(New_Co2))*100
        All_MISSES = All_MISSES+MISS_NUM[0]
        All_Lengths = All_Lengths+len(New_Co2)
        
        Device_new = np.stack((TIME_1_i,New_Co2,New_temperature,New_humidity,New_light,New_motion,New_battery,New_sound_avg,New_sound_peak,New_moisture,New_pressure,MISS_NUM,Error),axis=-1)
        Device_new_df = pd.DataFrame(data = Device_new,columns = ['Time','Co2','Temperature','Humidity','Light','Motion','Battery','Sound_avg','Sound_peak','Moisture','Pressure','Misses','Error'])
        Device_new_df.to_csv('Dev_' + Devices_Headers_1[i] + '_new.csv', index=True, header=True, sep=',')

In addition, we calculate the percentage of error for each device. We save the new file as csv file with the ID of the device_new. Therefore, we have a folder with the original file which has the ID of the device and a new file where we detect the failed transmissions with the name ID_new